### This script extract EEG signals based on the EMG signals, in order to align SEEG signals to the movement;

Input: Not re-referenced proprocessing1.mat from matlab. 

Output: trigger extracted from the EMG signals.



In [3]:
import numpy as np
import matplotlib.pyplot as plt
from gesture.config import *
import hdf5storage

2024-07-18 13:58:13 - Start Program

pre_all: Running from CMD.


In [ ]:
from gesture.utils import sub_sf_dict
info_dict=sub_sf_dict()

In [ ]:
sids=[int(i) for i in list(info_dict.keys())]
sid=sids[0]
print(str('sid:'+str(sid)+'.'))
data_folder = data_dir + 'preprocessing/' + 'P' + str(sid) + '/'
data_path = data_folder + 'preprocessing1.mat'
mat = hdf5storage.loadmat(data_path)
datas = mat['Datacell']
good_channels = mat['good_channels']
channelNum = len(np.squeeze(good_channels))
# channelNum=int(mat['channelNum'][0,0])
# total channel = channelNum + 4[2*emg + 1*trigger_indexes + 1*emg_trigger]
#data = np.concatenate((data[0, 0], data[0, 1]), 0)

In [ ]:
info_dict

### loop through sessions

In [ ]:
session=1
data=datas[0,session-1] # chn + emg1 +emg2 + emgdiff + trigger_label
BCIdata = data[:,:-4]
EMG=data[:,-4:-2]
EMGdiff=data[:,-2]
trigger_indexes = data[:,-1]

In [ ]:
%matplotlib inline
fig,ax=plt.subplots()
ax.plot(trigger_indexes)
plt.show()

In [ ]:
fig,ax=plt.subplots(3,1)
ax[0].plot(EMG[:,0])
ax[1].plot(EMG[:,1])
ax[2].plot(EMGdiff)
plt.show()

In [ ]:
%matplotlib inline
#%matplotlib notebook
fig,ax=plt.subplots()
plt.plot(EMGdiff[:100000])
plt.show()

In [ ]:
len(trigger_indexes)

In [ ]:
ch_types = ['eeg',]*BCIdata.shape[1]
ch_names = ['eeg'+str(i) for i in range(BCIdata.shape[1])]
info = mne.create_info(ch_names=ch_names, sfreq=1000, ch_types=ch_types)
raw_eeg = mne.io.RawArray(BCIdata.transpose(), info)

In [ ]:
ch_types = ['eeg','eeg','eeg']
ch_names = ['emg1','emg2','emgdiff']
info = mne.create_info(ch_names=ch_names, sfreq=1000, ch_types=ch_types)
raw_emg = mne.io.RawArray(np.concatenate((EMG.transpose(),EMGdiff[np.newaxis,:])), info)

In [ ]:
%matplotlib inline
raw_eeg.compute_psd().plot() #TODO: check the wierd PSD around 350-430hz and a bump at 470hz.

In [ ]:
freqs = (50, 100, 150, 200, 250, 300, 350, 400, 450)
raw_eeg.notch_filter(freqs=freqs)

In [ ]:
%matplotlib inline
raw_emg.compute_psd().plot()

In [ ]:
freqs = (50, 150, 250, 350, 450)
raw_emg.notch_filter(freqs=freqs)
raw_emg.filter(90,145)
raw_emg.compute_psd().plot()

In [ ]:
%matplotlib qt
emg=raw_emg.get_data()
emg_diff=emg[0,:]-emg[1,:]

In [ ]:
ch_types = ['eeg']
ch_names = ['emgdiff']
info = mne.create_info(ch_names=ch_names, sfreq=1000, ch_types=ch_types)
raw_emgdiff = mne.io.RawArray(emg_diff[:,np.newaxis].transpose(), info)
env=raw_emgdiff.copy().apply_hilbert(envelope=True).get_data()
env=np.squeeze(env)

In [ ]:
# smooth the envelope
from scipy.signal import savgol_filter
env_smooth = savgol_filter(env, 300, 5) #  window size 13, polynomial order 5
env_smooth_deriv=[]
for i in range(0,len(env_smooth)-1):
    #env_smooth_deriv.append(env_smooth[i]-2*env_smooth[i-1]+env_smooth[i-2])
    env_smooth_deriv.append(abs(env_smooth[i]-env_smooth[i-1]))
env_smooth_deriv.append(0)

In [ ]:
ind = np.where(trigger_indexes > 0)[0]
plt.plot(emg_diff)
plt.plot(env)
plt.plot(env_smooth)
plt.plot([i*5 for i in env_smooth_deriv])
for i in ind:
    plt.axvline(x=i, color='green', linestyle='--')


![](img\img1.png)

### manually identify the EMG onset point

In [ ]:
data_all=np.concatenate((raw_eeg.get_data(),EMG.transpose(),raw_emgdiff.get_data(),env[np.newaxis,:],
                         np.asarray(env_smooth_deriv)[np.newaxis,:],trigger_indexes[np.newaxis,:]),axis=0)

ch_types = ['eeg',]*len(raw_eeg.ch_names)+['misc',]*6
ch_names = ['eeg'+str(i) for i in range(len(raw_eeg.ch_names))]+['emg','emg','emg_diff','emg_diff_env','env_smooth_deriv','trigger_index']
info = mne.create_info(ch_names=ch_names, sfreq=1000, ch_types=ch_types)
raw_all = mne.io.RawArray(data_all, info)

In [ ]:
from gesture.utils import get_good_channels
tmin=-1
tmax=1
events_eeg = mne.find_events(raw_all, stim_channel='trigger_index')
epochs_eeg = mne.Epochs(raw_all, events_eeg, tmin=tmin, tmax=tmax, baseline=None)
            
good_channels = get_good_channels()
good_channel = good_channels['sid' + str(sid)]  # start from 1 (Matlab)
good_channel = [i - 1 for i in good_channel]
bad_channels_ind = [i for i in range(len(epochs_eeg.ch_names) - 6) if i not in good_channel]
bad_channels = [epochs_eeg.ch_names[c] for c in bad_channels_ind]
epochs_eeg.load_data()
epochs_eeg.drop_channels(bad_channels)

In [ ]:
epoch3=epochs_eeg['3'].load_data().pick(picks=[ch_names[78], 'emg-0', 'emg-1', 'emg_diff', 'emg_diff_env','env_smooth_deriv','trigger_index'])
epoch=epoch3.copy().filter(l_freq=0.5,h_freq=2,picks=['eeg'])
data=epoch.get_data() #(20, 6, 501)


In [ ]:
%matplotlib qt
trial=6
plt.plot([i*50 for i in data[trial,0,:]]) # EEG
plt.plot(data[trial,3,:]) # emg_diff
plt.plot(data[trial,4,:]) # env

### Locate movement start from the EMG signals

In [ ]:
# based on the env_smooth (15)
ind_emg=[]
threshold=3
for i in ind:
    j=0
    base_smooth1=env_smooth[i-1000:i]
    base_deriv2=env_smooth_deriv[i-1000:i]
    mean1 = np.mean(base_smooth1)
    std1 = np.std(base_smooth1)
    mean2 = np.mean(base_deriv2)
    std2 = np.std(base_deriv2)
    
    while True:
        value1=env_smooth[i+j]
        value2=env_smooth_deriv[i+j]
        z_score1 = (value1 - mean1) / std1
        z_score2 = (value2 - mean2) / std2
        if value1 > mean1*1.5 and abs(z_score1) > threshold and abs(z_score2) > threshold:
            ind_emg.append(i+j)
            break
        j=j+1

Result not good. This method is too sensitive.
Below is epoch created by the EMG trigger, shown alongside with the original triggers on the left.
Solution: add threshold requirement.

![](img\9.png)

In [ ]:
plt.plot(emg_diff)
plt.plot(env)
plt.plot(env_smooth)
plt.plot([i*5 for i in env_smooth_deriv])
for i,j in zip(ind,ind_emg):
    plt.axvline(x=i, color='green', linestyle='--')
    plt.axvline(x=j, color='red', linestyle='--')

![](img/img2.png)

### create new trigger index based on the EMG trigger

In [ ]:
sequence=[int(i) for i in trigger_indexes if i != 0]
trigger_ind_emg=np.zeros(trigger_indexes.shape)
for i,j in enumerate(ind_emg):
    trigger_ind_emg[j]=sequence[i]

In [ ]:
# compare EEG trigger and EMG trigger
%matplotlib inline
fig,ax=plt.subplots()
ax.plot(trigger_indexes,linestyle='-')
ax.plot(trigger_ind_emg,linestyle='--')
ax.legend(['trigger','trigger_emg'])
plt.show()

### combine all data into raw_all: raw_eeg(121),EMG(2),raw_emgdiff2(1),emgdiff_env(1),env_smooth_deriv(1),trigger_indexes(1),trigger_ind_emg(1)
channel number=EEG channels 121+7=128

In [ ]:
#raw_eeg.get_data()-(121, 590096),EMG-(590096, 2)
# ,raw_emgdiff2- (1, 590096),emgdiff_env-(1, 590096),env_smooth_deriv-(590096,),trigger_indexes-(590096,),trigger_ind_emg-(590096,)
data_all=np.concatenate((raw_eeg.get_data(),EMG.transpose(),raw_emgdiff.get_data(),env[np.newaxis,:],
                         np.asarray(env_smooth_deriv)[np.newaxis,:],trigger_indexes[np.newaxis,:],trigger_ind_emg[np.newaxis,:]),axis=0)

In [ ]:
data_all.shape

In [ ]:
ch_types = ['eeg',]*len(raw_eeg.ch_names)+['misc',]*7
ch_names = ['eeg'+str(i) for i in range(len(raw_eeg.ch_names))]+['emg','emg','emg_diff','emg_diff_env','env_smooth_deriv','trigger_index','trigger_index_emg']
info = mne.create_info(ch_names=ch_names, sfreq=1000, ch_types=ch_types)
raw_all = mne.io.RawArray(data_all, info)

In [ ]:
# save raw_all
data_folder = data_dir + 'preprocessing_no_re_ref/' + 'P' + str(sid) + '/'
filename=data_folder+'emg_trigger_raw.fif'
raw_all.save(filename,overwrite=True)

In [ ]:
filename

### checking the epoch result

In [ ]:
from gesture.group_analysis.util import get_epoch_and_anat_label
check_this_channel=78
tmin=-1
tmax=1
sf=1000
epochs=get_epoch_and_anat_label(sid, sf,tmin=tmin,tmax=tmax,trigger='EMG',re_ordered=True,random_shift=False)
#epochs=get_epoch(sid, sf,scaler='no',trigger='EMG',tmin=tmin,tmax=tmax)
epoch3=epochs['3'].load_data().pick(picks=[ch_names[check_this_channel], 'emg-0', 'emg-1', 'emg_diff', 'emg_diff_env','env_smooth_deriv','trigger_index','trigger_index_emg']) # simple grasp

In [ ]:
epoch=epoch3.copy().filter(l_freq=0.5,h_freq=2,picks=['eeg'])
data=epoch.get_data() #(20, 6, 501)

In [ ]:
# check individual trials
%matplotlib qt
fig,ax=plt.subplots()
trial_num=[0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,]
for i in trial_num:
    ax.plot([j+i*600 for j in data[i,0,:]])
    ax.plot([j+i*600 for j in data[i,3,:]*0.2])
    ax.plot([j*100+i*600 for j in data[i,-1,:]])
    ax.plot([j*100+i*600 for j in data[i,-2,:]])
filename=data_dir+'/group_analysis/phase_locking_sid'+str(sid)+'.png'
#fig.savefig(filename)
#fig.clear()
#plt.close(fig)
print('Phase-locking.')